# Спринт 4: рекомендации соавторов

In [1]:
import pandas as pd
from collections import Counter, defaultdict

# Read/Clean data

In [2]:
path = '../input/cndbv13/test1.json'
df = pd.read_json(path, lines=True, chunksize = 450000)
df = next(iter(df))
df.shape

(450000, 20)

In [3]:
# Оставим только необходимые для решения задачи поля
use_cols = ['_id', 'title', 'authors', 'venue', 'year', 'n_citation', 'lang', 'volume', 'issue', 'pdf', 'url', 'fos', 'references', 'keywords', 'abstract']

In [4]:
df = df[use_cols]

In [5]:
df.shape
df.drop_duplicates(['_id'], keep='last').shape
df['authors'] = df['authors'].astype('str')
df.drop_duplicates(['title', 'authors'], keep='last').shape
df = df.drop_duplicates(['title', 'authors'], keep='last')

# Recommendation System


Для решения задачи рекомендации соавторов для автора был предпринят "исторический" подход.  
Суть метода состоит в том, что автору следует рекомендовать тех и только тех, кто уже публиковался с ним ранее.  
На взгляд автора кода, именно такой подход будет наиболее успешно моделировать паттерны поведение в научном сообществе:  
1. Рассмотрим простой случай: студент выполняет работу в паре с научным руководителем/аспирантом - далее в подавляющем большинстве случаев они идут как соавторы в публикациях.
2. Другой пример - работа научного коллектива в рамках лаборатории. Как правило, коллектив разделен на подгруппы, специализирующихся на разных задачах. Либо же работа "размазана" между всеми (например, один получает образцы, дгугой делает замеры и вносит в базу данных, третий - моделирует и так далее). В любом случае, работа публикается одной и той же группой/подгруппой.
3. Регулярное сотрудничесво между научными организациями.  
В любом из приведенных выше случаев данный простой подход выдаст правдоподобные рекомендации. 
Естесвенно, при таком подходе мы можем легко ошибится; и даже более - ошибаться постоянно, если тот или иной автор кардинально сменил вид сферы деятельности. Однако, будем считать, что доля таких случаев невелика.

In [6]:
import json
from tqdm import tqdm
from typing import List, Union
from collections import defaultdict

In [7]:
def initilize_coauth(buffer: List[str], res: dict) -> dict:
    buffer = []
    for row_idx, row in enumerate(str_):
        info_arr = sorted(row.split(','))
        
        if "_id" in info_arr[0] and "name" in info_arr[1]:
            buffer.append(info_arr)

    co_ath = [(1, elem[1][5:]) for elem in buffer[1:]]
    res[buffer[0][0][4:]] = co_ath
    return res

In [8]:
def get_name_id(values: list, name: str) -> int:
    names = list(map(lambda x: x[1], values))
    try:
        return names.index(name)
    except ValueError as err:
        return

In [9]:
vals = [(1, 'MarkusKrötzsch'), (1, 'DennyVrandecic'), (1, 'HeikoHaller'), (1, 'RudiStuder}')]
name = 'HeikoHaller'
print(get_name_id(vals, name))

2


In [10]:
def update_coauth(buffer: List[str], res: dict, idx: str) -> dict:
    buffer = []
    for row_idx, row in enumerate(str_):
        info_arr = sorted(row.split(','))
        
        if "_id" in info_arr[0] and "name" in info_arr[1]:
            buffer.append(info_arr)
    
    for couple in buffer:
        upgr_idx = get_name_id(res[idx], couple[1][5:])
        if upgr_idx is not None:
            res[idx][upgr_idx][0] += 1
            res[idx].sort()
    return res

In [11]:
authors = df['authors']
res = defaultdict(int)
for idx in tqdm(range(len(authors) - 1)):
    try:
        str_ = authors[idx].split('}')
    except KeyError:
        continue
    if '_id\':' not in authors[idx]:
        continue
    str_ = authors[idx].replace('\'', '').replace('[', '').replace(']', '').replace('{', '').replace(' ', '').split('},')
    buffer = []
    try:
        for row_idx, row in enumerate(str_):
            info_arr = sorted(row.split(','))

            if "_id" in info_arr[0] and "name" in info_arr[1]:
                buffer.append(info_arr)
                
        for k in range(len(buffer)):            
            main_author = buffer[k][0][5:]
    
            res = update_coauth(buffer, res, idx) if main_author in res else initilize_coauth(buffer, res)
    except IndexError:
        continue
#     co_ath = [[1, elem[1][5:]] for elem in buffer[1:]]
#     res[buffer[0][0]] = co_ath

100%|██████████| 449804/449804 [00:17<00:00, 25799.05it/s]


Сохраним полученные результаты в json

In [14]:
with open('recommendations.json', 'w') as f:
    json.dump(res, f)  

In [ ]:
def get_recommendation(authors_links: defaultdict, author_id: str, 
                       num_recommendations=2) -> Union[list, str]:
    recds = []
    if authors_links[author_id]:
        authors_available = authors_links[author_id]
        recds = [author[1] for author in authors_available[:max(
               len(authors_available),
               num_recommendations)]]
        return recds            
       
    elif authors_links[author_id] == []:
        return f'Author {author_id} does not need the recommendation of any co-author'
        
    else:
        return 'No recommendation available'        

**Посмотрим примеры**  
Ниже приведены 3 случая:  
1. Автор ранее опубликовал одну и более статью с другими авторами  
2. Автор либо не имеет статей, либо не указывал соавторов ни в одной из статей  
3. Автора с запрашиваемым id отсутсвие в списке

In [ ]:
get_recommendation(res, '5489bbdadabfaed7b5fa3df8')

In [ ]:
get_recommendation(res,'53f47915dabfaefedbbb728f')

In [ ]:
get_recommendation(res,'53f47915dabfaefedbbb7281')